In [1]:
import os
import jieba
from collections import Counter
import re

In [2]:
# settings
n = 3  # 3-gram

data_path       = '../2_预处理/data/'          # 存放预处理后新闻数据的目录
wordtable_path  = '../2_预处理/wordtable.txt'  # 词表路径
stopwords_path  = '../2_预处理/stopwords/中文停用词表.txt' # 停止词表路径
testset_path    = './testset/'       # 测试集的目录
prediction_path = './predictions/'   # 存放预测结果的目录

In [3]:
ngrams_list = []  # n元组（分子）
prefix_list = []  # n-1元组（分母）

# 遍历所有预处理过的新闻文件
for i, datafile in enumerate(os.listdir(data_path)):
    with open(data_path + datafile, encoding='utf-8') as f:
        for line in f:
            sentence = ['<BOS>'] + line.split() + ['<EOS>']  # 列表，形如：['<BOS>', '显得', '十分', '明亮', '<EOS>']
            ngrams = list(zip(*[sentence[i:] for i in range(n)]))   # 一个句子中n-gram元组的列表
            prefix = list(zip(*[sentence[i:] for i in range(n-1)])) # 历史前缀元组的列表
            ngrams_list += ngrams
            prefix_list += prefix

ngrams_counter = Counter(ngrams_list)
prefix_counter = Counter(prefix_list)

In [4]:
all_words = []  # 词表中的全部词
with open(wordtable_path, encoding='utf-8') as f:
    for line in f.readlines()[1:]:
        all_words.append(line.split()[-1])

In [5]:
# 停止词
with open(stopwords_path) as f:
    stopwords = f.readlines()
stopwords = set(map(lambda x:x.strip(), stopwords))  # 去除末尾换行符

In [6]:
def probability(sentence):
    """
    计算一个句子的概率。
    Params:
        sentence: 由词构成的列表表示的句子。
    Returns:
        句子的概率。
    """
    prob = 1  # 初始化句子概率
    ngrams = list(zip(*[sentence[i:] for i in range(n)]))   # 将句子处理成n-gram的列表
    for ngram in ngrams:
        # 累乘每个n-gram的概率，并使用加一法进行数据平滑
        prob *= (1 + ngrams_counter[ngram]) / (len(prefix_counter) + prefix_counter[(ngram[0], ngram[1])])
    return prob

In [7]:
def predict(pre_sentence, post_sentence, all_words, cand_num=1):
    """
    根据历史进行一个词的预测。
    Params:
        pre_sentence: 待预测词之前部分句子的分词结果构成的列表。
        post_sentence: 待预测词之后部分句子的分词结果构成的列表。
        all_words: 所有候选词构成的列表。
        cand_num: 候选词数，默认为1。
    Returns:
        一个含有cand_num个元素的列表，表示预测的词，概率由大到小排序；
        如果预测失败，返回None。
    """
    word_prob = []  # 候选词及其概率构成的元组的列表
    for word in all_words:
        # 实际上不需要算整个句子的概率，只需要算待预测词附近的概率即可，因为句子其他部分的概率不受待预测词影响
        test_sentence = pre_sentence[-(n-1):] + [word] + post_sentence[:(n-1)]  # 待预测词及其前后各n-1个词的列表
        word_prob.append( (word, probability(test_sentence)) )                  # (词, 概率)元组构成的列表

    return sorted(word_prob, key=lambda tup: tup[1], reverse=True)[:cand_num]  # 按概率降序排序并取前cand_num个

In [8]:
# 加载测试集标签（答案）
with open('testset/answer.txt', encoding='utf-8') as f:
    answers = [answer.strip() for answer in f]  # 答案构成的列表
    
prediction_file = open(prediction_path + 'prediction_ngram.txt', 'w', encoding='utf-8')  # 存放预测结果

# 开始测试
correct_count = 0  # 预测正确的数量

with open('testset/questions.txt', encoding='utf-8') as f:
    questions = f.readlines()  # 测试集规模
    total_count = len(questions)
    for i, question in enumerate(questions):
        question = question.strip()
        pre_mask = question[:question.index('[MASK]')]     # 待预测词的历史
        post_mask = question[question.index('[MASK]')+6:]  # 待预测词后的剩余部分
        
        pre_sentence = jieba.cut(pre_mask.replace('，', ' '))  # 分词
        post_sentence = jieba.cut(post_mask.replace('，', ' '))  # 分词
        pre_sentence = [word.strip() for word in pre_sentence if word.strip() and word not in stopwords]  # 去除停止词、空串
        post_sentence = [word.strip() for word in post_sentence if word.strip() and word not in stopwords]  # 去除停止词、空串

        predict_cand = predict(pre_sentence, post_sentence, all_words)  # 预测一个概率最大的词
        prediction_file.write(' '.join([w[0] for w in predict_cand]) + '\n')  # 将预测结果写入文件

        # 遍历多个预测结果
        for j, p in enumerate(predict_cand):
            if p[0] == answers[i]:
                print(i, '{} [{}] {}'.format(pre_mask, p[0], post_mask))
                correct_count += 1
                break
                    
prediction_file.close()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/8l/s96gb4dx64sbmzrfs718s78r0000gn/T/jieba.cache
Loading model cost 0.653 seconds.
Prefix dict has been built succesfully.


0 一直以来，有不少家长误认为，在开车时将孩子放在儿童安全 [座椅] 上是最为安全的方式。
5 体验人员在驴妈妈平台体验预订景点门票时，发现 [故宫] 门票均绑定了内馆门票、导览、讲解等收费项目，没有单卖的故宫门票供选择，涉嫌强制消费。
11 总而言之，从大环境上来说，目前仍然是处于教育用户的阶段，加上扫地 [机器人] 品类的下限很低，市场还没有形成一个足够清晰的认知。
12 TOF技术虽然在手机应用上占据了大量的市场，但大多数 [应用] 由于比较“鸡肋”而难以支撑其进一步发展。
14 在谈到与特斯拉的竞争时，小鹏汽车高管表示，小鹏没有照搬特斯拉的系统，但特斯拉系统中很好的部分会去 [学习] ，与此同时特斯拉踩过坑我们可以避免。
18 在三大运营商积极展开5G部署工作的同时，各大手机 [厂商] 也在积极布局5G产品，说到这里就不能不提到OPPO了。
29 学生禁带手机进校园的规定落实之初，被抓到玩手机的学生会由 [班主任] 谈话，没收的手机交给家长。
36 随后李国庆微博发长文回应，称俞渝对我私生活做出的 [诽谤] 和诬蔑，我只想在这里回应一句话：等着收律师函吧。
39 对大多数人来说，再花钱购买另一项订阅 [服务] 可能听起来并不是很有吸引力，但苹果押注其初创公司云集的内容，即使是最节俭的客户也会被说服。
52 9月10日晚间消息，阿里20周年年会今日举行，马云登台发言正式 [宣布] 卸任阿里巴巴董事局主席。
79 安信证券研报指出，随着资本市场对人工智能认知的不断深入，市场对 [人工智能] 的投资日趋成熟和理性，投融资频次在2018年以来有所放缓，但投资金额持续增加。
81 上海移动的工作人员也向记者表示，暂时没有听说停止销售该类套餐。目前三大运营商中，只有中国电信明确将停售达量限速 [套餐] 。


In [9]:
print('准确率：{}/{}'.format(correct_count, total_count))

准确率：12/100
